<a href="https://colab.research.google.com/github/ingridmidory/D-a-de-2025/blob/main/Entrenamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenando un modelo de CNN

El siguiente modelo de redes neuronales convolucionales fue tomado del siguiente github:

> **[Clasificador-de-Perros-y-Gatos](https://github.com/Gamapro/Clasificador-de-Perros-y-Gatos)**

Este modelo se entrenó con los datos de entrenamiento y validación que creador del mismo nos proporciona en el github mencionado.

 Este trabajo fue realizado en el marco de la celebración del 🎉 ***Día Internacional de las Matemáticas*** 🎉 que se llevará a cabo el día 14 de marzo del 2025 en las instalaciones de la Escuela Nacional Preparatoria No. 4. Nuestro objetivo es acercar la IA a más personas de una manera práctica y divertida.

🔹 Créditos
* Modelo de CNN: Creado por [David Gamaliel Arcos](https://github.com/Gamapro)
* Participantes:
 * [Ingrid Monterroso Alfaro ](https://github.com/ingridmidory)
 * Ahuacatitan Espindola José Luis
 * De la Cruz Saavedra Uriel
 * Godoy Santiago José Ángel



In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from time import time
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Definir rutas en Google Drive

ruta_drive = "/content/drive/MyDrive/Diadepi /dataset"  # Cambia esto según el nombre de tu carpeta
data_entrenamiento = "/content/drive/MyDrive/Diadepi /entrenamiento" #importante
data_validacion = "/content/drive/MyDrive/Diadepi /validacion" #importante

In [ ]:
# Parametros

epocas = 20
altura, longitud = 100,100
batch_size = 32
batch_size_val = 16
pasos = 500
pasos_validacion = 100
filtrosConv1 = 8
filtrosConv2 = 16
filtrosConv3 = 32
tamano_filtro1 = (8,8)
tamano_filtro2 = (4,4)
tamano_filtro3 = (2,2)
pool = (2,2)
clases = 2
lr = 0.001

In [ ]:
# Preprocesamiento

entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,  # Cada pixel este entre 0 y 1
    shear_range = 0.2, # Generar las imagenes "inclinadas"
    #zoom_range = 0.2,  # Zoomearlas parahacer "enfasis"
    horizontal_flip = True   # Invertir para distinguir direccionalidad
)

validacion_datagen = ImageDataGenerator(
    rescale = 1./255  # Solo reescalado para las imagenes de test
)

imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size = (altura, longitud),
    #classes = ['perro','gato'],
    batch_size = batch_size,
    class_mode = 'categorical'
)

imagen_validacion = validacion_datagen.flow_from_directory(
    data_validacion,
    target_size = (altura, longitud),
    #classes = ['perro','gato'],
    batch_size = batch_size_val,
    class_mode = 'categorical'
)

#print(imagen_entrenamiento.class_indices)
def plots(ims, figsize=(20,20), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if(ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims)%2==0 else len(ims)//rows+1
    for i in range(len(ims)):
        sp = f.add_subplot(rows,cols,i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i],interpolation=None if interp else "none")
    plt.show()
imgs, labels = next(imagen_entrenamiento)
#plots(imgs,titles=labels)

Found 980 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [ ]:
# Crear la Red Neuronal Convolucional
cnn = Sequential()

# primera capa
cnn.add(Conv2D(filtrosConv1, tamano_filtro1, padding='same', input_shape=(altura,longitud,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=pool))

# Segunda Capa
cnn.add(Conv2D(filtrosConv2, tamano_filtro2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=pool))

# Tercera Capa
cnn.add(Conv2D(filtrosConv3, tamano_filtro3, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=pool))

# Volverla plana, Cuarta y Quinta capa
cnn.add(Flatten())
cnn.add(Dense(4, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

# Compilar el modelo
cnn.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Entrenar el modelo
log_dir = "logs/{}".format(time())
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=0)

cnn.fit(
    imagen_entrenamiento,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=imagen_validacion,
    validation_steps=pasos_validacion,
    callbacks=[tensorboard_callback]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 31/500 ━━━━━━━━━━━━━━━━━━━━ 4:15 545ms/step - accuracy: 0.5325 - loss: 0.6952

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


500/500 ━━━━━━━━━━━━━━━━━━━━ 472s 936ms/step - accuracy: 0.5279 - loss: 0.6935 - val_accuracy: 0.4988 - val_loss: 0.6933
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 157s 313ms/step - accuracy: 0.5357 - loss: 0.6923 - val_accuracy: 0.5050 - val_loss: 0.6932
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 136s 180ms/step - accuracy: 0.5348 - loss: 0.6918 - val_accuracy: 0.5013 - val_loss: 0.6934
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 85s 66ms/step - accuracy: 0.5337 - loss: 0.6916 - val_accuracy: 0.5006 - val_loss: 0.6936
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 37s 58ms/step - accuracy: 0.5347 - loss: 0.6913 - val_accuracy: 0.5050 - val_loss: 0.6936
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - accuracy: 0.5340 - loss: 0.6912 - val_accuracy: 0.4950 - val_loss: 0.6946
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 70s 56ms/step - accuracy: 0.5346 - loss: 0.6910 - val_accuracy: 0.5088 - val_loss: 0.6935
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 42s 83ms/step - accuracy: 0.5342 - loss: 0.6910 - val_a

In [ ]:
# Guardar el modelo en Google Drive
modelo_dir = ruta_drive + "modelo/"
if not os.path.exists(modelo_dir):
    os.mkdir(modelo_dir)

cnn.save(modelo_dir + "modelo_perros_gatos.h5")
cnn.save_weights(modelo_dir + "pesos_perros_gatos.weights.h5") # Changed the file extension to .weights.h5

print("¡Entrenamiento completado y modelo guardado en Google Drive!")

¡Entrenamiento completado y modelo guardado en Google Drive!
